# Xopt CNSGA algorithm

In [ ]:
from xopt.generators.ga.cnsga import CNSGAGenerator

from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs

from xopt.utils import read_xopt_csv

from xopt import Xopt, Evaluator

import pandas as pd

from glob import glob

import matplotlib.pyplot as plt

In [ ]:
# Useful for debugging
# %load_ext autoreload
# %autoreload 2

In [ ]:
ev = Evaluator(function=evaluate_TNK)
ev.function_kwargs = {
    "raise_probability": 0.1
}  # optional random crashing, to mimic real-world use.

In [ ]:
X = Xopt(
    generator=CNSGAGenerator(vocs=tnk_vocs),
    evaluator=ev,
    vocs=tnk_vocs,
)
X.strict = False

Run 100 generations

In [ ]:
%%time
for _ in range(64 * 20):
    X.step()

# Plot 

In [ ]:
def plot_population(X):
    fig, ax = plt.subplots(figsize=(8, 8))

    fdata = tnk_vocs.feasibility_data(X.data)

    k1 = "x1"
    k2 = "x2"

    X.data.plot.scatter(k1, k2, marker=".", alpha=0.1, color="black", ax=ax)
    X.data[fdata["feasible"]].plot.scatter(
        k1, k2, marker="x", alpha=0.3, color="orange", ax=ax
    )
    X.generator.population.plot.scatter(k1, k2, marker="o", color="red", alpha=1, ax=ax)
    ax.set_xlabel(k1)
    ax.set_ylabel(k2)
    ax.set_xlim(0, 1.5)
    ax.set_ylim(0, 1.5)
    ax.set_title("TNK with Xopt's CNSGA")

In [ ]:
plot_population(X)

Write the current population

In [ ]:
X.generator.write_population("test.csv")

# YAML method

In [ ]:
YAML = """
max_evaluations: 6400
strict: False
generator:
    name: cnsga
    population_size: 64
    population_file: test.csv
    output_path: .

evaluator:
    function: xopt.resources.test_functions.tnk.evaluate_TNK
    function_kwargs:
      raise_probability: 0.1

vocs:
    variables:
        x1: [0, 3.14159]
        x2: [0, 3.14159]
    objectives: {y1: MINIMIZE, y2: MINIMIZE}
    constraints:
        c1: [GREATER_THAN, 0]
        c2: [LESS_THAN, 0.5]
    constants: {a: dummy_constant}

"""

X = Xopt(YAML)
X

This will have loaded children from the population file. These will need to be re-evaluated.

In [ ]:
len(X.generator._children)

In [ ]:
%%time
X.run()

In [ ]:
plot_population(X)

In [ ]:
len(X.data)

Setting `output_path` will write .csv files for each population, as well as the offspring considered in each generation

In [ ]:
pop_files = sorted(glob("cnsga_population*"))
pop_files[:10]

In [ ]:
offspring_files = sorted(glob("cnsga_offspring*"))
offspring_files[0:10]

In [ ]:
pop_df = read_xopt_csv(pop_files[-1])
pop_df.plot.scatter("x1", "x2", marker="o", color="red", alpha=1)

Similarly, offsrping files can be loaded. This will load the last few:

In [ ]:
offspring_df = read_xopt_csv(*offspring_files[-10:])
offspring_df.plot.scatter("x1", "x2", marker=".", color="black", alpha=0.1)

Occationally there are duplicates in offspring

In [ ]:
all_offspring = read_xopt_csv(*offspring_files)
len(all_offspring), len(all_offspring.drop_duplicates())

In [ ]:
# Cleanup
!rm cnsga_population*
!rm cnsga_offspring*
!rm test.csv

# Examine generator

In [ ]:
df = pd.DataFrame(X.generator.generate(1000))

fig, ax = plt.subplots()
df.plot.scatter(
    "x1", "x2", marker=".", color="green", alpha=0.5, ax=ax, label="candidates"
)
pop_df.plot.scatter(
    "x1", "x2", marker="o", color="red", alpha=1, ax=ax, label="population"
)
plt.legend()

# Vectorized evaluation

Some functions also allow vectorized inputs. This can often be very fast.

However, vectorized evaluation has some restrictions. For example, the output dict cannot append additional arrays with odd lengths.

In [ ]:
# Notice that this returns `some_array`
evaluate_TNK({"x1": 1, "x2": 1})

In [ ]:
# Here we make a version that does not have this
def evaluate_TNK2(*args, **kwargs):
    outputs = evaluate_TNK(*args, **kwargs)
    outputs.pop("some_array")
    return outputs

In [ ]:
YAML = """
max_evaluations: 6400
strict: False
generator:
    name: cnsga
    population_size: 64

evaluator:
    function: __main__.evaluate_TNK2
    function_kwargs:
      raise_probability: 0.1
    vectorized: True
    max_workers: 100 

vocs:
    variables:
        x1: [0, 3.14159]
        x2: [0, 3.14159]
    objectives: {y1: MINIMIZE, y2: MINIMIZE}
    constraints:
        c1: [GREATER_THAN, 0]
        c2: [LESS_THAN, 0.5]
    constants: {a: dummy_constant}

"""


X2 = Xopt.from_yaml(YAML)
X2.evaluator.function = evaluate_TNK2

X2.run()

len(X2.data)

In [ ]:
plot_population(X)